In [1]:
import numpy as np
import tensorflow as tf

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
n = 10

A = np.random.rand(10000, 10000).astype('float32')
B = np.random.rand(10000, 10000).astype('float32')

In [4]:
c1 = []

def matpow(M, n):
    if n < 1: 
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

In [5]:
with tf.device('/gpu:0'): # Para executar na CPU, use /cpu:0
    a = tf.constant(A)
    b = tf.constant(B)
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

In [6]:
with tf.device('/cpu:0'): # Para executar na CPU, use /cpu:0
    a = tf.constant(A)
    b = tf.constant(B)
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

In [11]:
# Input
input_value = tf.constant(0.5, name="input_value")

# Peso
weight = tf.Variable(1.0, name="weight")

# Output esperado (usado no treinamento)
expected_output = tf.constant(0.0, name="expected_output")

with tf.GradientTape() as tape:
    # Criação do modelo
    model = tf.multiply(input_value, weight, "model")

    # Cost Function
    loss_function = tf.pow(model - expected_output, 2, name="loss_function")
gradient = tape.gradient(loss_function, [weight])

# Otimizador
optimizer = tf.optimizers.SGD(learning_rate=0.01)
optimizer.apply_gradients(zip(gradient, [weight]))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [13]:
writer = tf.summary.create_file_writer("./summary")
with writer.as_default():
    for i, value in enumerate([input_value, weight, expected_output, model, loss_function]):
        tf.summary.scalar(f'Tensor: {i}', value, step=i)
        writer.flush()    
        